## Módulo 17 - t-SNE

In [1]:
# EDA and data visualization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# statistics and maths
import numpy as np
from scipy.stats import gaussian_kde

# ML / tSNE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.manifold import TSNE

### Bloco 1 - AED

#### Carga de dados 

Neste vídeo, iniciamos um projeto prático utilizando o algoritmo t-SNE para visualização de dados. Foram instaladas e importadas diversas bibliotecas, como pandas, seaborn e scikit-learn. Carregamos um conjunto de dados de cosméticos em um DataFrame, explorando sua estrutura e conteúdo. Observamos as colunas com informações sobre tipos de cosméticos, marcas, preços, ratings e ingredientes. O próximo passo será explorar mais a fundo os dados e avançar para as próximas etapas do projeto.

In [2]:
df_cosmetics = pd.read_csv("../datasets/raw/dataset_cosmetics.csv")
df_cosmetics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tipo           1472 non-null   object 
 1   Marca          1472 non-null   object 
 2   Nome           1472 non-null   object 
 3   Preco          1472 non-null   int64  
 4   Rating         1472 non-null   float64
 5   Ingredientes   1472 non-null   object 
 6   Pele_Mista     1472 non-null   int64  
 7   Pele_Seca      1472 non-null   int64  
 8   Pele_Normal    1472 non-null   int64  
 9   Pele_Oleosa    1472 non-null   int64  
 10  Pele_Sensivel  1472 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 126.6+ KB


**features**:

- Categoria: Categoria do Produto
- Marca: Marca do Produto
- Nome: Nome do Produto
- Preco: PreÃ§o do Produto
- Rating: Rating do Produto
- Ingredientes: Lista de Ingredientes / Componentes
- Pele_Mista: Serve para Pele Mista?
- Pele_Seca: Serve para Pele Seca?
- Pele_Normal: Serve para Pele Normal?
- Pele_Oleosa: Serve para Pele Oleosa?
- Pele_Sensivel: Serve para Pele SensÃ­vel?

In [3]:
df_cosmetics.head(4)

,Tipo,Marca,Nome,Preco,Rating,Ingredientes,Pele_Mista,Pele_Seca,Pele_Normal,Pele_Oleosa,Pele_Sensivel
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1


#### Análise Univariada

Neste trecho da aula prática, foi abordada a Análise Exploratória de Dados (EDA), com foco na análise univariada de um DataFrame. Foi discutido como lidar com variáveis de texto, transformando-as em listas para posterior análise estatística. Em seguida, foi realizada uma análise estatística das variáveis do DataFrame, destacando a distribuição de valores entre categorias, como tipo e marca. Foi demonstrado como criar gráficos de barras para visualizar as informações de forma mais clara, destacando as principais marcas e tipos de produtos.

Na análise univariada, exploramos a variável "ingrediente" em um DataFrame separado, identificando 7.298 ingredientes diferentes, o que pode causar problemas de dimensionalidade. Em seguida, analisamos a distribuição das variáveis numéricas "preço" e "rating", destacando a concentração de valores entre US$35 e US$49 para o preço e a predominância de notas altas, principalmente em torno de 4.3 e 4.4, para o rating. Essa etapa encerra a primeira parte da Análise Exploratória de Dados (EDA).

In [4]:
# transform column's values 'ingredients' in a new Dataframe
df_cosmetics_eda = df_cosmetics.copy()
df_cosmetics_eda = df_cosmetics_eda['Ingredientes'].str.split(',')
df_cosmetics_eda

0       [Algae (Seaweed) Extract,  Mineral Oil,  Petro...
1       [Galactomyces Ferment Filtrate (Pitera),  Buty...
2       [Water,  Dicaprylyl Carbonate,  Glycerin,  Cet...
3       [Algae (Seaweed) Extract,  Cyclopentasiloxane,...
4       [Water,  Snail Secretion Filtrate,  Phenyl Tri...
                              ...                        
1467    [Water,  Alcohol Denat.,  Potassium Cetyl Phos...
1468    [Water,  Isododecane,  Dimethicone,  Butylocty...
1469    [Water,  Dihydroxyacetone,  Glycerin,  Scleroc...
1470    [Water,  Dihydroxyacetone,  Propylene Glycol, ...
1471                      [Visit the DERMAFLASH boutique]
Name: Ingredientes, Length: 1472, dtype: object

In [5]:
# create ingredients dataframe
df_ingredients = df_cosmetics_eda.explode('Ingredientes')
df_ingredients

0              Algae (Seaweed) Extract
1                          Mineral Oil
2                           Petrolatum
3                             Glycerin
4                        Isohexadecane
                     ...              
45811           Alpha-isomethyl Ionone
45812                 CI 14700 (Red 4)
45813              CI 19140 (Yellow 5)
45814               CI 42090 (Blue 1).
45815    Visit the DERMAFLASH boutique
Name: Ingredientes, Length: 45816, dtype: object

In [6]:
# statitistics features
df_cosmetics.describe()

,Preco,Rating,Pele_Mista,Pele_Seca,Pele_Normal,Pele_Oleosa,Pele_Sensivel
count,1472.000000,1472.000000,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000
mean,55.584239,4.153261,0.65625,0.614130,0.652174,0.607337,0.513587
std,45.014429,0.633918,0.47512,0.486965,0.476442,0.488509,0.499985
min,3.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,4.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,42.500000,4.300000,1.00000,1.000000,1.000000,1.000000,1.000000
75%,68.000000,4.500000,1.00000,1.000000,1.000000,1.000000,1.000000
max,370.000000,5.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [7]:
# PLOT DE DISTRIBUIÇÃO POR TIPO DE PRODUTO

# Calcula o percentual por tipo
percentual_tipo = df_cosmetics.value_counts('Tipo') / len(df_cosmetics) * 100
percentual_tipo = percentual_tipo.reset_index()
percentual_tipo.columns = ['Tipo', 'Percentual']

# Cria o gráfico de barras
fig = px.bar(percentual_tipo, x='Tipo', y='Percentual', color='Tipo',
             text='Percentual', color_discrete_sequence=px.colors.sequential.Peach)

# Formata os rótulos de texto e o layout
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Atualiza o layout para um design premium
fig.update_layout(
    title={
        'text': "Distribuição Percentual por Tipo de Produto",
        'y': 0.95,  # Posição vertical
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Tipo de Produto',
    yaxis_title='Percentual (%)',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        categoryorder='total descending',  # Ordena de forma decrescente
        tickangle=-45  # Rotaciona os labels do eixo X para legibilidade
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='lightgrey'
    ),
    template='plotly_white',  # Um template elegante e clean
    # Ajusta as margens para evitar que os labels sejam cortados
    margin=dict(l=60, r=50, t=100, b=100),
    height=600,  # Ajusta o tamanho do gráfico
    width=900,   # Largura maior para melhor visualização
    showlegend=False  # Remover legenda
)

# Ajusta o hover (informações ao passar o mouse)
fig.update_traces(
    hovertemplate='<b>%{x}</b><br>Percentual: %{y:.2f}%',
    marker_line_width=1.5,  # Aumenta a borda das barras
    marker_line_color="black"  # Cor da borda
)

# Adiciona anotação para o maior valor
max_val = percentual_tipo['Percentual'].max()
max_tipo = percentual_tipo[percentual_tipo['Percentual']
                           == max_val]['Tipo'].values[0]
fig.add_annotation(
    x=max_tipo,
    y=max_val,
    text=f'Maior valor: {max_val:.2f}%',
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowcolor="green",
    ax=0,
    ay=-40,
    font=dict(size=12, color="green"),
    bgcolor="lightgreen",
    bordercolor="green",
    borderwidth=1.5
)

# Exibe o gráfico
fig.show()

In [8]:
# PLOT DE DISTRIBUIÇÃO PERCENTUAL POR MARCA

# Calcula o percentual por marca
percentual_marca = df_cosmetics.value_counts('Marca') / len(df_cosmetics) * 100
percentual_marca = percentual_marca.reset_index()
percentual_marca.columns = ['Marca', 'Percentual']

# Seleciona as 10 maiores marcas
percentual_marca = percentual_marca.nlargest(10, 'Percentual')

# Cria o gráfico de barras horizontal
fig = px.bar(percentual_marca, x='Percentual', y='Marca', color='Marca',
             text='Percentual', orientation='h',
             color_discrete_sequence=px.colors.sequential.Blues_r)

# Formata os rótulos de texto e o layout
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Atualiza o layout para um design premium
fig.update_layout(
    title={
        'text': "Distribuição Percentual das 10 Maiores Marcas de Cosméticos",
        'y': 0.95,  # Posição vertical
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Percentual (%)',
    yaxis_title='Marca',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        tickfont=dict(size=14),
        categoryorder='total ascending'  # Ordena de forma ascendente no eixo Y
    ),
    template='plotly_white',  # Um template elegante e clean
    # Ajusta as margens para evitar que os labels sejam cortados
    margin=dict(l=100, r=50, t=100, b=50),
    height=600,  # Ajusta o tamanho do gráfico
    width=900,   # Largura maior para melhor visualização
    showlegend=False  # Remover legenda
)

# Ajusta o hover (informações ao passar o mouse)
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>Percentual: %{x:.2f}%',
    marker_line_width=1.5,  # Aumenta a borda das barras
    marker_line_color="black"  # Cor da borda
)

# Adiciona anotação para o maior valor
max_val = percentual_marca['Percentual'].max()
max_marca = percentual_marca[percentual_marca['Percentual']
                             == max_val]['Marca'].values[0]
fig.add_annotation(
    x=max_val,
    y=max_marca,
    text=f'Maior valor: {max_val:.2f}%',
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowcolor="blue",
    ax=40,
    ay=0,
    font=dict(size=12, color="blue"),
    bgcolor="lightblue",
    bordercolor="blue",
    borderwidth=1.5
)

# Exibe o gráfico
fig.show()

In [9]:
# quantidade total de marcas
len(df_cosmetics['Marca'].unique())

116

In [10]:
# PLOT DE DISTRIBUIÇÃO PERCENTUAL POR INGREDIENTE

# Calcula o percentual por ingrediente
percentual_ingredients = df_ingredients.value_counts(
    'Ingredientes') / len(df_ingredients) * 100
percentual_ingredients = percentual_ingredients.reset_index()
percentual_ingredients.columns = ['Ingredientes', 'Percentual']

total_ingredients = len(percentual_ingredients['Ingredientes'].unique())

# Seleciona os 10 ingredientes mais frequentes
percentual_ingredients = percentual_ingredients.nlargest(10, 'Percentual')

# Cria o gráfico de barras horizontal
fig = px.bar(percentual_ingredients, x='Percentual', y='Ingredientes', color='Ingredientes',
             text='Percentual', orientation='h',
             color_discrete_sequence=px.colors.sequential.Sunset)

# Formata os rótulos de texto e o layout
# Mais casas decimais e "nano%" como sufixo
fig.update_traces(texttemplate='%{text:.4s}%', textposition='outside')

# Atualiza o layout para um design mais profissional
fig.update_layout(
    title={
        'text': "Distribuição Percentual dos 10 Ingredientes Mais Usados",
        'y': 0.95,  # Posição vertical
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Percentual',
    yaxis_title='Ingrediente',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        tickfont=dict(size=14),
        categoryorder='total ascending'  # Ordena de forma ascendente no eixo Y
    ),
    template='plotly_white',  # Um template elegante e clean
    # Ajusta as margens para evitar que os labels sejam cortados
    margin=dict(l=120, r=50, t=100, b=50),
    height=600,  # Ajusta o tamanho do gráfico
    width=900,   # Largura maior para melhor visualização
    showlegend=False  # Remover legenda
)

# Ajusta o hover (informações ao passar o mouse)
fig.update_traces(
    hovertemplate='<b>%{y}</b><br>Percentual: %{x:.4s}%',
    marker_line_width=1.5,  # Aumenta a borda das barras
    marker_line_color="black"  # Cor da borda
)

# Exibe o gráfico
fig.show()

In [11]:
# quantidade total de ingredientes
total_ingredients

7298

In [12]:
# DISTRIBUIÇÃO DA VARIÁVEL PREÇO

# Dados do preço
preco_data = df_cosmetics['Preco']

# Cria o histograma
fig = px.histogram(preco_data, nbins=30, color_discrete_sequence=['#636EFA'])

# Adiciona uma linha de densidade (KDE)
fig.add_trace(go.Scatter(
    x=np.linspace(preco_data.min(), preco_data.max(), 100),
    y=np.histogram(preco_data, bins=30, density=True)[0],
    mode='lines',
    name='Densidade',
    line=dict(color='red', width=2)
))

# Adiciona linha de preço médio e mediano
preco_medio = preco_data.mean()
preco_mediano = preco_data.median()

# Linha de preço médio
fig.add_vline(x=preco_medio, line_dash="dash", line_color="green", 
               annotation_text="Preço Médio", annotation_position="top right",
               annotation=dict(font=dict(color="green")))

# Linha de preço mediano - reposicionada abaixo
fig.add_vline(x=preco_mediano, line_dash="dot", line_color="blue", 
               annotation_text="Preço Mediano", annotation_position="bottom right",
               annotation=dict(font=dict(color="blue")))

# Atualiza o layout
fig.update_layout(
    title={
        'text': "Distribuição da Variável Preço com Curva de Densidade",
        'y':0.95,  # Posição vertical
        'x':0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Preço (R$)',
    yaxis_title='Frequência',
    xaxis_title_font=dict(size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True, 
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True, 
        gridcolor='lightgrey'
    ),
    template='plotly_white',  # Um template elegante e clean
    margin=dict(l=60, r=50, t=100, b=100),  # Ajusta as margens
    height=600,  # Tamanho do gráfico
    width=900,   # Largura do gráfico
    showlegend=True,  # Exibe a legenda
    legend=dict(
        orientation="h",  # Horizontal
        yanchor="bottom",  # Ancla a legenda na parte inferior
        y=-0.2,           # Posiciona abaixo do gráfico
        xanchor="center",  # Centraliza
        x=0.5              # Ajusta para o centro
    )
)

# Customiza o hover para exibir informações detalhadas
fig.update_traces(hovertemplate='Preço: R$%{x}<br>Frequência: %{y}')

# Exibe o gráfico
fig.show()


In [13]:
# Dados do rating
rating_data = df_cosmetics['Rating'].dropna()  # Remove valores nulos, se houver

# Cria o histograma
fig = px.histogram(rating_data, nbins=30, color_discrete_sequence=['#636EFA'])

# Cálculo da densidade (KDE)
kde = gaussian_kde(rating_data)
x_values = np.linspace(rating_data.min(), rating_data.max(), 100)
kde_values = kde(x_values)

# Adiciona a linha de densidade (KDE) ao gráfico
fig.add_trace(go.Scatter(
    x=x_values,
    y=kde_values * rating_data.size * (x_values[1] - x_values[0]),  # Ajusta a escala da densidade
    mode='lines',
    name='Densidade',
    line=dict(color='red', width=2)
))

# Adiciona linha de rating médio e mediano
rating_medio = rating_data.mean()
rating_mediano = rating_data.median()

# Linha de rating médio
fig.add_vline(x=rating_medio, line_dash="dash", line_color="green", 
               annotation_text="Rating Médio", annotation_position="top right",
               annotation=dict(font=dict(color="green")))

# Linha de rating mediano - reposicionada abaixo
fig.add_vline(x=rating_mediano, line_dash="dot", line_color="blue", 
               annotation_text="Rating Mediano", annotation_position="bottom right",
               annotation=dict(font=dict(color="blue")))

# Atualiza o layout
fig.update_layout(
    title={
        'text': "Distribuição da Variável Rating com Curva de Densidade",
        'y': 0.95,  # Posição vertical
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Rating',
    yaxis_title='Frequência',
    xaxis_title_font=dict(size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True, 
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True, 
        gridcolor='lightgrey'
    ),
    template='plotly_white',  # Um template elegante e clean
    margin=dict(l=60, r=50, t=100, b=100),  # Ajusta as margens
    height=600,  # Tamanho do gráfico
    width=900,   # Largura do gráfico
    showlegend=True,  # Exibe a legenda
    legend=dict(
        orientation="h",  # Horizontal
        yanchor="bottom",  # Ancla a legenda na parte inferior
        y=-0.2,           # Posiciona abaixo do gráfico
        xanchor="center",  # Centraliza
        x=0.5              # Ajusta para o centro
    )
)

# Customiza o hover para exibir informações detalhadas
fig.update_traces(hovertemplate='Rating: %{x}<br>Frequência: %{y}')

# Exibe o gráfico
fig.show()


#### Análise Bivariada

Neste trecho da aula, abordamos a análise bivariada de um conjunto de dados de cosméticos. Exploramos a relação entre o tipo de produto e o preço, utilizando boxplot e identificando outliers. Em seguida, analisamos a relação entre tipo e rating dos produtos, destacando variações nos valores. Também examinamos a correlação entre preço e rating por tipo, utilizando um scatterplot. Por fim, apresentamos uma matriz de correlação para avaliar possíveis relações fortes entre as variáveis numéricas do DataFrame.

In [14]:
# PLOT DE DISTRIBUIÇÃO (BOX PLOT POR TIPO E PREÇO)

# Cria o box plot
fig = px.box(df_cosmetics,
             x='Preco',
             y='Tipo',
             color='Tipo',
             orientation='h',
             hover_data=['Marca'],
             # Cores agradáveis e distintas
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Atualiza o layout
fig.update_layout(
    title={
        'text': "Distribuição do Preço por Tipo de Produto",
        'y': 0.95,  # Posição vertical do título
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Preço (R$)',
    yaxis_title='Tipo de Produto',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey',  # Cor das linhas de grade
        zerolinecolor='darkgrey'  # Cor da linha zero
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey',  # Cor das linhas de grade
        zerolinecolor='darkgray'  # Cor da linha zero
    ),
    plot_bgcolor='whitesmoke',  # Fundo do gráfico em cinza claro
    paper_bgcolor='white',  # Fundo do papel
    template='plotly_white',  # Um template elegante e clean
    margin=dict(l=60, r=50, t=100, b=100),  # Ajusta as margens
    height=600,  # Tamanho do gráfico
    width=900,   # Largura do gráfico
)

# Exibe o gráfico
fig.show()

In [15]:
# Cria o box plot
fig = px.box(df_cosmetics,
             x='Rating',
             y='Tipo',
             color='Tipo',
             orientation='h',
             hover_data=['Marca'],
             # Cores agradáveis e distintas
             color_discrete_sequence=px.colors.qualitative.Plotly)

# Atualiza o layout
fig.update_layout(
    title={
        'text': "Distribuição do Rating por Tipo de Produto",
        'y': 0.95,  # Posição vertical do título
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Rating',
    yaxis_title='Tipo de Produto',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey',  # Cor das linhas de grade
        zerolinecolor='darkgrey'  # Cor da linha zero
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey',  # Cor das linhas de grade
        zerolinecolor='darkgrey'  # Cor da linha zero
    ),
    plot_bgcolor='whitesmoke',  # Fundo do gráfico em cinza claro
    paper_bgcolor='white',  # Fundo do papel
    template='plotly_white',  # Um template elegante e clean
    margin=dict(l=60, r=50, t=100, b=100),  # Ajusta as margens
    height=600,  # Tamanho do gráfico
    width=900,   # Largura do gráfico
)

# Exibe o gráfico
fig.show()

In [16]:
# correlação preço e Rating
# Cria o gráfico de dispersão
fig = px.scatter(
    df_cosmetics,
    x='Preco',
    y='Rating',
    color='Tipo',  # Cores por tipo de produto
    hover_data=['Marca'],  # Informações adicionais ao passar o mouse
    # Adiciona uma linha de tendência (Ordinary Least Squares)
    trendline='ols',
    color_discrete_sequence=px.colors.qualitative.Plotly  # Paleta de cores
)

# Atualiza o layout
fig.update_layout(
    title={
        'text': "Correlação entre Preço e Rating dos Produtos",
        'y': 0.95,  # Posição vertical do título
        'x': 0.5,   # Centraliza o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, family='Arial, sans-serif', color='#333'),
    xaxis_title='Preço (R$)',
    yaxis_title='Rating',
    xaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    yaxis_title_font=dict(
        size=18, family='Arial, sans-serif', color='#4B4B4B'),
    xaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey'  # Cor das linhas de grade
    ),
    yaxis=dict(
        tickfont=dict(size=12),
        showgrid=True,
        gridcolor='darkgrey'  # Cor das linhas de grade
    ),
    plot_bgcolor='whitesmoke',  # Fundo do gráfico em cinza claro
    paper_bgcolor='white',  # Fundo do papel
    template='plotly_white',  # Um template elegante e clean
    margin=dict(l=60, r=50, t=100, b=100),  # Ajusta as margens
    height=600,  # Tamanho do gráfico
    width=900,   # Largura do gráfico
)

# Exibe o gráfico
fig.show()

In [17]:
# Gerar a matriz de correlação
matrix_corr_cosmetics = df_cosmetics.corr(numeric_only=True)

# Criar o heatmap interativo com a escala de cores RdYlBu
fig = px.imshow(
    matrix_corr_cosmetics,
    color_continuous_scale='RdYlBu',  # Escala de cores
    zmin=-1,
    zmax=1,
    labels=dict(x='Variáveis', y='Variáveis', color='Correlação'),
    title='Matriz de Correlação - Cosméticos',
    aspect='auto'  # Ajustar a proporção
)

# Adicionar anotações com cores de texto dinâmicas
for i in range(len(matrix_corr_cosmetics)):
    for j in range(len(matrix_corr_cosmetics)):
        # Define a cor do texto com base no valor da correlação
        correlation_value = matrix_corr_cosmetics.iloc[i, j]
        # Texto branco para valores fortes
        text_color = 'white' if abs(correlation_value) > 0.5 else 'black'

        fig.add_annotation(
            x=j,
            y=i,
            text=f"{correlation_value:.3f}",
            showarrow=False,
            font=dict(color=text_color, size=10)
        )

# Exibir o gráfico
fig.show()

### Bloco 2 - Treinamento do Modelo

#### Preparação dos Dados para Modelo

Neste trecho da aula, continuamos o projeto prático com o treinamento do algoritmo t-SNE. Foi feita uma preparação dos dados, onde foi criada uma cópia do DataFrame original e algumas colunas foram removidas. Foram separadas variáveis numéricas e categóricas, aplicando transformadores como `StandardScaler` e `OneHotEncoder`. Foi criado um PreProcessor para aplicar essas transformações nos dados. A importância do parâmetro Remainder foi destacada para preservar colunas não transformadas. Ingredientes não foram usados devido à alta dimensionalidade. O próximo passo será rodar o algoritmo e armazenar os dados para visualização.

In [18]:
# copiar dataframe original com menos variáveis
X = df_cosmetics.copy()
X.drop(columns=['Nome', 'Ingredientes'], axis=1, inplace=True)

# separar variáveis
numeric_features = ['Rating', 'Preco']
categorical_features = ['Marca', 'Tipo']

# definir transformações
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# criar preprocessador de transformação
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# tranformar os dados
X_transformed = preprocessor.fit_transform(X)

# visualizar resumo dos dados preprocessados
X_transformed

<1472x129 sparse matrix of type '<class 'numpy.float64'>'
	with 10368 stored elements in Compressed Sparse Row format>

#### Treinamento do Modelo

Neste trecho, é explicado como criar uma rotina para treinar um modelo várias vezes, armazenar os resultados e visualizá-los em um gráfico. É mencionado o uso do algoritmo t-SNE, a variação do parâmetro perplexity de 5 a 50, a criação de um dataframe para armazenar os resultados e a execução de um loop para treinar o modelo com diferentes valores de perplexity. Ao final, é mencionada a preparação do dataframe para a visualização dos resultados em um próximo vídeo.

In [30]:
# armazenar os resultados do t-SNE em Dataframe
results_df = pd.DataFrame()

In [31]:
for perplexity in range(5, 51, 1):

    # criar e treinar o modelo
    tsne = TSNE(
        n_components=2,
        perplexity=perplexity,
        init='random',
        n_iter=250,
        random_state=51,
    )
    tsne_results = tsne.fit_transform(X_transformed)

    # armazenar resultados
    temp_df = pd.DataFrame(
        tsne_results, columns=['Component_1', 'Component_2'])
    temp_df['Perplexity'] = perplexity
    results_df = pd.concat([results_df, temp_df], axis=0)

In [32]:
results_df.head()

,Component_1,Component_2,Perplexity
0,-0.717949,-0.395937,5
1,-0.816794,-0.376122,5
2,0.865159,0.020383,5
3,-0.730861,-0.385952,5
4,1.143533,-0.364589,5


In [33]:
results_df.reset_index(drop=True, inplace=True)
results_df.head()

,Component_1,Component_2,Perplexity
0,-0.717949,-0.395937,5
1,-0.816794,-0.376122,5
2,0.865159,0.020383,5
3,-0.730861,-0.385952,5
4,1.143533,-0.364589,5


### Bloco 3 - Análise dos Resultados

#### Visualização dos Resultados em 2D

Neste projeto prático, aprendemos a criar um scatterplot animado para visualizar os resultados do algoritmo t-SNE com diferentes valores de perplexity. A animação mostra como os dados se comportam conforme o perplexity aumenta, destacando a formação de clusters. Essa técnica é útil para explorar e visualizar dados de forma mais dinâmica, permitindo uma análise mais aprofundada e a tomada de decisões embasadas. Essa abordagem nos ajuda a compreender a importância dos hiperparâmetros na análise de dados.

In [39]:
# criar um Scatterplot animado com variação no Perplexity

# scatterplot
fig = px.scatter(
    results_df,
    x='Component_1',
    y='Component_2',
    animation_frame='Perplexity',
    title='Visualização t-SNE: Redução Dimensional com Variação de Perplexity',
    labels={
        'Component_1': 'Componente 1',
        'Component_2': 'Componente 2',
        'Perplexity': 'Perplexidade'
    },
    template='plotly_white'  # Estilo mais limpo e profissional
)

# layout
fig.update_layout(
    title={
        'text': 'Visualização t-SNE: Redução Dimensional com Variação de Perplexidade',
        'y': 0.9,  # Posição vertical do título
        'x': 0.5,  # Posição horizontal do título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font_size=24,  # Aumentar o tamanho do título para dar maior destaque
    xaxis_title='Componente 1',  # Título eixo X
    yaxis_title='Componente 2',  # Título eixo Y
    # Linhas discretas do grid
    xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='LightGray'),
    # Linhas discretas do grid
    yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='LightGray'),
    font=dict(
        size=14,  # Aumentar tamanho da fonte para facilitar a leitura
        # Alterar o tipo de fonte para algo mais elegante e profissional
        family="Arial, sans-serif",
    ),
    # Ajustar margens para equilíbrio visual
    margin=dict(l=40, r=40, t=80, b=40),
    legend=dict(
        x=1.05,  # Colocar a legenda fora da área do gráfico para maior clareza
        y=1,
        bgcolor='rgba(255, 255, 255, 0.5)',  # Fundo semitransparente
        bordercolor="Black",
        borderwidth=1
    ),
    transition_duration=500,  # Animação mais fluida, 500ms por frame
    plot_bgcolor='whitesmoke',  # Fundo do gráfico em Whitesmoke
)

# Melhorar o estilo dos marcadores
fig.update_traces(
    # Marcadores com bordas discretas
    marker=dict(size=8, opacity=0.8, line=dict(
        width=1, color='DarkSlateGray')),
)

# Mostrar o gráfico
fig.show()

#### Visualização dos Resultados em 3D

Neste resumo, demonstramos como adaptar um código de visualização de 2D para 3D. Exploramos a adição de um terceiro componente ao DataFrame e a utilização do método scatter 3D do Plotly para exibir os dados em um cubo. Apesar da visualização em 3D não destacar tão claramente os agrupamentos quanto a visualização em 2D, ressaltamos a possibilidade de explorar o 3D em outros contextos. Concluímos que, para esse caso específico, a visualização em 2D é mais informativa.